# PySpark Fundamentals

### List of Topics
1. Initializing SparkSession
2. 

#### Initializing SparkSession

In [41]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

#### Create DataFrame

In [42]:
from datetime import datetime, date
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

In [43]:
df.printSchema()


root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [ ]:
schema='a long, b double, c string, d date, e timestamp'
data = [
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
]

df = spark.createDataFrame(data=data, schema=schema)

#### View Dataframe Contents

In [ ]:
df.show()

In [ ]:
df.show(vertical=True)

In [ ]:
df.describe().show()

#### collect(), take(), tail()

` DataFrame.collect(): Collects ALL the distributed data to the driver side as the local data in Python
> df.collect()


` DataFrame.take(n): Collects First n number of distributed data to the driver side as the local data in Python
> df.take(2)

` DataFrame.tail(n): Collects First n number of distributed data to the driver side as the local data in Python
> df.tail(2)

In [ ]:
df.collect()

In [ ]:
df.take(2)


In [ ]:
df.tail(2)

#### CSV

In [ ]:
df.write.csv('temp/foo.csv', header=True)

In [ ]:
df = spark.read.csv('temp/foo.csv', header=True)
df.show()

#### Parquet

In [44]:
df.write.parquet('/temp/bar.parquet')



23/02/07 22:53:38 ERROR FileOutputCommitter: Mkdirs failed to create file:/temp/bar.parquet/_temporary/0
23/02/07 22:53:39 ERROR Executor: Exception in task 7.0 in stage 51.0 (TID 180)
java.io.IOException: Mkdirs failed to create file:/temp/bar.parquet/_temporary/0/_temporary/attempt_202302072253383288453009389202975_0051_m_000007_180 (exists=false, cwd=file:/Volumes/D/Workspace/DATA/Databricks/pyspark)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:329)
	at org.apache.parquet.hadoop.ParquetOutputFormat.getRecordWriter(ParquetOutputFormat.java:482)
	at org.apache.parquet.hadoop.ParquetOu

Py4JJavaError: An error occurred while calling o190.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org